In [35]:
import requests
import time
import pandas as pd
import numpy as np

# bL_lat = -8.84892992
# bL_lon = 35.74657448
# tR_lat = 6.49954908
# tR_lon = 44.78973632
#These coordinates go from the bottom left of the canary islands 
# to the top right of Spain and the balearic islands, ensuring we 
# capture all mobile towers in a single API call

def get_data_from_api(bL_lat, bL_lon, tR_lat, tR_lon):
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'sec-ch-ua-platform': '"Windows"',
        'Accept': '*/*',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://geoportal.minetur.gob.es/VCTEL/vcne.do',
        'Accept-Language': 'en-GB-oxendict,en;q=0.9,es-ES;q=0.8,es;q=0.7',
    }
    params = (
        ('idCapa', 'null'),
        ('bbox', f'{bL_lat}, {bL_lon}, {tR_lat}, {tR_lon}'), 
        ('zoom', '4'),
    )
    response = requests.get('https://geoportal.minetur.gob.es/VCTEL/infoantenasGeoJSON.do', headers=headers, params=params)
    return pd.Series(response.json())

In [36]:
def scrape_data_from_given_area(start_lat, start_lon, end_lat, end_lon, lat_jump, lon_jump):
    current_lat = start_lat
    current_lon = start_lon
    accum_data = pd.Series() 
    start_time = time.time()
    errorAt = pd.Series()
    while current_lon < end_lon:
        while current_lat < end_lat:
            a = get_data_from_api(current_lat, current_lon, current_lat + lat_jump, current_lon + lon_jump)
            print(current_lat, current_lon, end='\r')
            if a.empty == False:
                try:
                    print(a, end='\r')
                    accum_data.append(get_data_from_api(current_lat, current_lon, current_lat + lat_jump, current_lon + lon_jump), ignore_index=False, verify_integrity=False)
                except:
                    errorAt.append([current_lat, current_lon])
                    pass
            current_lat += lat_jump
        pcg_cmn_dec = round((current_lon - start_lon+0.1) / (end_lon - start_lon),2)
        pcg_cmn_str = str(pcg_cmn_dec*100) + '%'
        current_time = time.time()
        elapsed_time_hrs = (current_time+0.001 - start_time)/60/60
        time_left_hrs = (elapsed_time_hrs / pcg_cmn_dec)
        print(pcg_cmn_str, str(round(time_left_hrs,2))+'hrs left')
        current_lat = start_lat
        current_lon += lon_jump
    return accum_data

In [37]:
start_lat = -8.84892992
start_lon = 35.846574479999994
end_lat = 6.49954908
end_lon = 43.69657447999955 #new end_lon (altura punto mas alto de españa, en galicia)
lat_jump = 0.05
lon_jump = 0.05
# jumps=0.5 --> 55,779 mobile towers
# jumps=0.4 --> 57,958 mobile towers
# jumps=0.35 --> 58,116 mobile towers
# jumps=0.3 --> 59,561 mobile towers
# jumps=0.2 --> 62,072 mobile towers
# jumps=0.1 --> 65,252 mobile towers
# jumps=0.05 --> never managed to do the whole thing without an error, but 65.000 is better than what AM got. 
# When you learn how to save stuff in SQL database try this again, it will restart each time where you left off!

mobile_towers_data = scrape_data_from_given_area(start_lat, start_lon, end_lat, end_lon, lat_jump, lon_jump)

<ipython-input-36-5f40cedd8c93>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  accum_data = pd.Series()
<ipython-input-36-5f40cedd8c93>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  errorAt = pd.Series()
<ipython-input-35-77ee9258189c>:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(response.json())


type                                        FeatureCollection
features    [{'type': 'Feature', 'geometry': {'type': 'Poi...
type                                        FeatureCollection
features    [{'type': 'Feature', 'geometry': {'type': 'Poi...
1.0% 2.04hrs left 35.8465744799999944444
type                                        FeatureCollection
features    [{'type': 'Feature', 'geometry': {'type': 'Poi...
type                                        FeatureCollection
features    [{'type': 'Feature', 'geometry': {'type': 'Poi...
2.0% 2.41hrs left 35.896574479999999999
3.0% 2.51hrs left 35.946574479999999999
type                                        FeatureCollection
features    [{'type': 'Feature', 'geometry': {'type': 'Poi...
type                                        FeatureCollection
features    [{'type': 'Feature', 'geometry': {'type': 'Poi...


KeyboardInterrupt: 

In [38]:
mobile_towers_data[:3]

NameError: name 'mobile_towers_data' is not defined

In [26]:
## Filter out items that are empty dictionaries
def checkIfFull(item):
    if item == dict():
        return False  
    return True 
non_empty_items_iterator = filter(checkIfFull, mobile_towers_data)
non_empty_numbers = list(non_empty_items_iterator)
mobile_towers_data = non_empty_numbers
mobile_towers_data[:3]

NameError: name 'mobile_towers_data' is not defined

In [ ]:
# Remove unwanted keys from the 5 seperate dictionaries 
# (we currently have a list of 5 dictionaries, each dictionary having 2 subdictionaries, one of which is useless)
new_mobile_towers_data = []
for item in mobile_towers_data:
    new_mobile_towers_data.append(item['features'])
new_mobile_towers_data[:3]

In [ ]:
#Flatten the list of lists produced
flat_list = [item for sublist in new_mobile_towers_data for item in sublist]
#flat_list[:3]
print(len(flat_list), "mobile towers in scraped data")

In [43]:
#Create and open csv file with the useful data retrieved
import csv
import os

data_file = open('mobile_towers4.csv', 'w', newline='')
csv_writer = csv.writer(data_file)
 
count = 0
for data in flat_list:
    if count == 0:
        header = data['properties'].keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data['properties'].values())
 
os.startfile("mobile_towers4.csv")
data_file.close()

In [16]:
os.remove("mobile_towers.csv")